# Model Optimization

## Feature engineering

Now, we will work on feature engineering to optimize our model.
We can divide our features in two types: Meta and Text based.

### Meta features
These will include all the features that we get from the user or information we can obtain given our understanding of the english language.

1. Post upvotes 
2. Days since request
3. User subs on (count of subreddits) 
4. User activity comments
5. User activity comments raop 
6. User posts reddit 
7. User posts raop 
8. Comment count 
9. User rate start 
10. User rate end 
11. Account age (days)
12. Day of the week
13. Day of the month
14. Month
15. Week of the year
16. Text was edited

### Text based features
These will include all the features that are related to frequency, svd, sentiment.

1. Text length (word count) 
2. Text compound sentiment 
3. Title length (word count)
4. Title sentiment
5. TFIDF
6. SVD

## Let's import libraries and load data

In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Data manipulation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# ML Scikit

# NLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string

## SVD
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

## Decision Trees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

# Regression model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

# KNN Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

# Feature handling
from sklearn.model_selection import train_test_split

# Reports
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from six import StringIO

#Visualizing Confusion Matrix using Heatmap
import seaborn as sns


In [3]:
# load the data
df = pd.read_csv('../data/interim/logit_sentiments.csv')

In [9]:
df.columns
df.shape

(4040, 19)

In [10]:
pd.isnull(df['requester_received_pizza']).sum() # remove one null data point in outcome variable

df = df[df['requester_received_pizza'].notna()]

# Confirm no null values remaining
pd.isnull(df['requester_received_pizza']).sum()
df.shape

(4040, 19)

# Split data to evaluate new features

We're using the baseline classifier - Logistic regression

In [11]:
# Data Handling for Model

X, Y = df.iloc[:,:-1], df.iloc[:,-1]

# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X.iloc[shuffle], Y.iloc[shuffle]

# Split into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1)
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.4, random_state=1)


In [12]:
# Let's see the different tests

print("Train X,y - ", X_train.shape, " ", y_train.shape)
print("Dev X,y   -  ", X_dev.shape, "  ", y_dev.shape)
print("Test X,y  -  ", X_test.shape, "  ", y_test.shape)
print("Total: ", X_train.shape[0] + X_dev.shape[0] +  X_test.shape[0])

Train X,y -  (3030, 18)   (3030,)
Dev X,y   -   (606, 18)    (606,)
Test X,y  -   (404, 18)    (404,)
Total:  4040


In [13]:
type(y_train)
df['requester_received_pizza'].isnull().sum()

0

# Pre-processing text with TFIDF vectorizer and SVD dimensionality reduction

Let's see first the text we are interested in analyzing.

In [14]:
# Check nan
# pd.isnull(X_train['request_text']).sum()
# X_train['request_text'].count()

# See text messages
X_train.loc[:,'request_text']

3957    Hello RAOP,\n\nfigured I'd give posting here a...
2163    Hi RAOP!\n\nMy two roommates and I are all ful...
2170    I've recently graduated university. I think I ...
2027    So, I didn't find out my wife was leaving me t...
483     (*throwaway acct*)\nI never thought I'd get to...
                              ...                        
1899    First time on Random Acts of Pizza, my wallet ...
3082    My son has recieved his meal, thanks veritas27...
2724    Hey, I'm a broke college student, would really...
2669    I live in Goodyear, AZ.\n\nI can receive pizza...
1992    Well, my rent is paid up until June, and [my c...
Name: request_text, Length: 3030, dtype: object

In [15]:
# Fills nan in text features
# Input: array, array name
# Output: fill array nan with 'the'
def fill_nan_text(texts, feature):
    texts = texts.replace(np.nan, 'the', regex=True)
    return texts

# Improve text tokenization
# Input: string
# Output: string without stop words, and applied
# lemmatization and stemming
def better_preprocessor(s):
    
    s = s.lower() # to lowercase
    
    #Stemming and lemmatization
    ps = PorterStemmer()
    lemmer = WordNetLemmatizer()
    words = word_tokenize(s)
    cleaned_words = [lemmer.lemmatize(ps.stem(word)) for word in words]
    
    #Remove stop words
    stop_words = set(stopwords.words('english'))
    new_cleaned_words = [w for w in cleaned_words if not w.lower() in stop_words]
    s = ''
    for word in new_cleaned_words:
        if word == '.' or word == '?' or word == ',':
            s = s + word
        else:
            s = s + ' ' + word
    
    return s

# Vectorizes the text data into separated tokens as features
# Input: array with text
# Output: print size of vocabulary
def create_text_features_no_processor(train):
    
    # Vectorization with empty preprocessing of text
    vectorizer = CountVectorizer(preprocessor= lambda x: x)
    X_train_counts_raw = vectorizer.fit_transform(train)
    print('Size of train Vocabulary with empty pre-processor is:', X_train_counts_raw.shape)

# Vectorizes the text data into separated tokens as features and applies better processor
# Input: array with text
# Output: print size of vocabulary
def create_text_features_with_processor(train, test):
    
    # Vectorization with better preprocessor
    # declare Vectorizer with counts of words
    tfidf_vectorizer = TfidfVectorizer(
                                       analyzer='word',
                                       token_pattern=r'\w{1,}',
#                                        min_df=3, 
                                       preprocessor = better_preprocessor,
                                       ngram_range=(1,3))

    #vectorizer on train data
    X_train_counts = tfidf_vectorizer.fit_transform(fill_nan_text(train,'request_text'))
    X_test = tfidf_vectorizer.transform(fill_nan_text(test,'request_text'))
    print('---------- \nSize of train Vocabulary with better preprocessor is:', X_train_counts.shape[1])
    
    #feature names
    feature_names = tfidf_vectorizer.get_feature_names()
#     print(len(feature_names))
    return tfidf_vectorizer,X_train_counts, X_test, feature_names

def svd_reduction(X_train_counts,X_test,svd_comps, feature_names):
    
    #SVD for dimensionality reduction
    svd = TruncatedSVD(n_components=svd_comps,n_iter=10,random_state=42)
    normalizer = Normalizer(copy=False) #Normalizing SVD results
    lsa = make_pipeline(svd, normalizer) #making pipeline for svd and normalizer

    ## reduced by SVD
    X_svd = lsa.fit_transform(X_train_counts)
    X_test_svd = lsa.transform(X_test)
    print('---------- \nSize of train Vocabulary with better preprocessor and SVD is:', X_svd.shape[1])
    
    #explained variance by SVD reduced dimensions
    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
    
    #extract feature names corresponding to the 1500 SVD components
    best_features = [feature_names[i] for i in svd.components_[0].argsort()[::-1]]
    #print(best_features[:1500])
    
    print(f"Train array: {X_svd.shape}\nTest array: {X_test_svd.shape}")
    
    #Convert to pandas df
#     df_orig = pd.DataFrame(X_train_counts.toarray(), columns=feature_names)
#     df = pd.DataFrame(df_orig, columns=best_features[:1500])
  
    return X_svd, X_test_svd

def clean_array(array):
    # Remove original text variables
    array.pop('request_title')
    array.pop('request_text')
    # X_dev.columns
    return array

# Merge meta and text data for Classification And Build Logisitc Regression and Random Forest Classifiers


In [19]:
# Constants

svd_components = 200 # tested with 200, 400, 600, 1000, 1500
cs=1 # tested with 0.01,0.1,1,10,100
depths = [2,3,4]
estimators = [15, 30, 80]
best=[0,0,0,0]

# Data Handling for Model

X, Y = df.iloc[:,:-1], df.iloc[:,-1]

# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X.iloc[shuffle], Y.iloc[shuffle]

# Split into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1)
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.4, random_state=1)


# Vectorize with Tfidf + SVD 
# Vectorize with TfIDF + SVD
count_vector, train_counts, test_counts, features = create_text_features_with_processor(
    X_train.loc[:,'request_text'], 
    X_dev.loc[:,'request_text'])

train_svd, test_svd = svd_reduction(train_counts, test_counts,
                                    svd_components,
                                   features)

# Clean text variables: title and request
X_train = clean_array(X_train)
X_dev = clean_array(X_dev)

print(f"Train meta: {X_train.shape} - Train Text SVD: {train_svd.shape}") 
print(f"Test meta: {X_dev.shape} - Test Text SVD: {test_svd.shape}")

X_train_mt = np.concatenate((X_train,train_svd), axis=1)
X_dev_mt = np.concatenate((X_dev,test_svd), axis=1)
print(f"Test meta + text: {X_train_mt.shape} - Test meta + text: {X_dev_mt.shape}")

# fit model with L2 Regularization
print("**************************")
print("Logistic regression")
lr_model = LogisticRegression(C=cs, max_iter= 1000, solver='liblinear',
                             multi_class="auto", penalty ='l2')

#print(X_train_mt.shape, y_train.shape)
y_train_binarized = [1 if x == True else 0 for x in y_train] ### Binarize to make it an accpetable input to fit model
y_dev_binarized = [1 if x == True else 0 for x in y_dev]
lr_model.fit(X_train_mt, y_train_binarized)

# predictions
y_pred = lr_model.predict(X_dev_mt)

# results
#     cnf_matrix = metrics.confusion_matrix(y_dev_binarized, y_pred)
# print(cnf_matrix)
#     print(classification_report(y_dev_binarized, y_pred))
report_log = classification_report(y_dev_binarized, y_pred, output_dict=True)
print("Accuracy for Logistic regression model is", round(report_log['accuracy'],4))
print("**************************")
print("Random forest classifier")

for d in depths:
    for e in estimators:
        # XG Boost classifier
        clf = xgb.XGBClassifier(max_depth=d, 
                                n_estimators=e, 
#                                 colsample_bytree=0.8,
#                                 subsample=0.8, 
#                                 nthread=10, 
                                learning_rate=0.1,
                                use_label_encoder=False,
                                eval_metric='mlogloss')
        clf.fit(X_train_mt, y_train_binarized)
        predictions = clf.predict_proba(X_dev_mt)
        if clf.score(X_dev_mt, y_dev_binarized) > best[0]:
            print (f'Accuracy (xg boost): {clf.score(X_dev_mt, y_dev_binarized):.2} - d={d} - e={e}')
            best[0] = clf.score(X_dev_mt, y_dev_binarized)
            best[1] = d
            best[2] = e
            best[3] = 'xg_boost'
        # Random Forest
        rfc = RandomForestClassifier(n_estimators=e, max_depth=d)
        rfc.fit(X_train_mt, y_train_binarized)
        if clf.score(X_dev_mt, y_dev_binarized) > best[0]:
            print (f'Accuracy (RF): {rfc.score(X_dev_mt, y_dev_binarized):.2} - d={d} - e={e}')
            best[0] = rfc.score(X_dev_mt, y_dev_binarized)
            best[1] = d
            best[2] = e
            best[3] = 'RF'
            
        # Ada Boost
        abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=d)
                                 , n_estimators=e, learning_rate=0.1)
        abc.fit(X_train_mt, y_train_binarized)
        if abc.score(X_dev_mt, y_dev_binarized) > best[0]:
            print(f'Accuracy (adaboost): {abc.score(X_dev_mt, y_dev_binarized):.2} - d={d} - e={e}')
            best[0] = rfc.score(X_dev_mt, y_dev_binarized)
            best[1] = d
            best[2] = e
            best[3] = 'ada'

print("**************************")
print(f"The best classifier is {best[3]} with acc={best[0]:.3} - d={best[1]} - e={best[2]}")

---------- 
Size of train Vocabulary with better preprocessor is: 196211
---------- 
Size of train Vocabulary with better preprocessor and SVD is: 200
Explained variance of the SVD step: 13%
Train array: (3030, 200)
Test array: (606, 200)
Train meta: (3030, 16) - Train Text SVD: (3030, 200)
Test meta: (606, 16) - Test Text SVD: (606, 200)
Test meta + text: (3030, 216) - Test meta + text: (606, 216)
**************************
Logistic regression
Accuracy for Logistic regression model is 0.8449
**************************
Random forest classifier
Accuracy (xg boost): 0.83 - d=2 - e=15
Accuracy (adaboost): 0.83 - d=2 - e=15
Accuracy (xg boost): 0.83 - d=2 - e=30
Accuracy (xg boost): 0.83 - d=3 - e=30
**************************
The best classifier is xg_boost with acc=0.835 - d=3 - e=30


## Multilayer Neural Network

We are building the 3 layer multilayer neural network. Number of neurons in 3 layers is same as number of features in our dataset


In [20]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(216,216,216), activation='logistic', solver='adam', max_iter=500)
mlp.fit(X_train_mt,y_train_binarized)
y_pred = mlp.predict(X_dev_mt)

# print(confusion_matrix(y_dev_binarized,y_pred))
# print(classification_report(y_dev_binarized,y_pred))

# Model Accuracy: how often does model classify correctly
print("Accuracy:",metrics.accuracy_score(y_dev_binarized, y_pred))


Accuracy: 0.8118811881188119


## Dense Neural Network

Next we tried deep neural network to further improve our model accuracy. 

As this is a binary classification problem framed as predicting the likelihood of a data row belonging to one of the two classes, e.g. the class that you assign the integer value 1, whereas the other class is assigned the value 0. 

Hyperparameters used: Output Layer Configuration: One node with a sigmoid activation unit. 
Loss Function: Binary Cross-Entropy


In [21]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

print(tf.__version__)

2.5.0


In [36]:
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K

K.clear_session() #reset keras session everytime we run the model

#Converting y from list to numpy array to feed in the model
y_train_binarized = np.array(y_train_binarized)
y_dev_binarized = np.array(y_dev_binarized)

# number of nodes is same as number of features
model = tf.keras.Sequential()
model.add(layers.Dense(216,activation="sigmoid")) 
model.add(layers.Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["acc"])
history = model.fit(X_train_mt, y_train_binarized, verbose=0, batch_size=50, epochs=100, 
                    validation_data=(X_dev_mt, y_dev_binarized))

# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 216)               46872     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 217       
Total params: 47,089
Trainable params: 47,089
Non-trainable params: 0
_________________________________________________________________


In [38]:
history.history['acc']
print("Accuracy last epoch")
history.history.get('acc')[-1] # printing model accuracy at last epoch

Accuracy last epoch


0.8943894505500793

# Conclusion

After trying various machine learning classifiers our model built using dense neural networks 
achieved the highest accuracy of 0.89. 
